<a href="https://colab.research.google.com/github/YoonJiHwan98/dacon_code/blob/main/%EC%8B%9D%EC%88%98_%EC%9D%B8%EC%9B%90_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
import datetime

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/TAVE 머신러닝 스터디/식수 인원 예측/train.csv')
test = pd.read_csv('/content/drive/MyDrive/TAVE 머신러닝 스터디/식수 인원 예측/test.csv')

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/TAVE 머신러닝 스터디/식수 인원 예측/sample_submission.csv')

# 결측치 확인


In [ ]:
train.info() # 결측치는 없다!

# 파생 변수 생성



In [ ]:
train['식사 인원 수'] = train['본사정원수']- train['본사휴가자수'] - train['본사출장자수'] - train['현본사소속재택근무자수']

In [ ]:
train

**코로나를 빠트릴 수 없을 것 같아서 사회적 거리두기 단계 변수만 넣었습니다** 

In [ ]:
train['일자'] = pd.to_datetime(train['일자'])

In [ ]:
for i in range(len(train)):
    if train['일자'][i]<datetime.datetime(2020,8,23):
        train.loc[i,'사회적거리두기 1단계']=0
        train.loc[i,'사회적거리두기 1.5단계']=0
        train.loc[i,'사회적거리두기 2단계 이상']=0
    elif datetime.datetime(2020,8,23)<=train['일자'][i]<datetime.datetime(2020,10,13):
        train.loc[i,'사회적거리두기 1단계']=0
        train.loc[i,'사회적거리두기 1.5단계']=0
        train.loc[i,'사회적거리두기 2단계 이상']=1
    elif datetime.datetime(2020,10,13)<=train['일자'][i]<datetime.datetime(2020,12,8):
        train.loc[i,'사회적거리두기 1단계']=1
        train.loc[i,'사회적거리두기 1.5단계']=0
        train.loc[i,'사회적거리두기 2단계 이상']=0
    elif datetime.datetime(2020,12,8)<=train['일자'][i]<datetime.datetime(2021,2,15):
        train.loc[i,'사회적거리두기 1단계']=0
        train.loc[i,'사회적거리두기 1.5단계']=0
        train.loc[i,'사회적거리두기 2단계 이상']=1
    elif datetime.datetime(2021,2,15)<=train['일자'][i]:
        train.loc[i,'사회적거리두기 1단계']=0
        train.loc[i,'사회적거리두기 1.5단계']=1
        train.loc[i,'사회적거리두기 2단계 이상']=0

# 타겟과 변수간의 관계 시각화 


***일자***



In [ ]:
train_mon = train.set_index('일자')

In [ ]:
train_mon= train_mon.resample('M').mean()
train_mon

In [ ]:
plt.figure(figsize=(15, 12))
sns.lineplot(data = train_mon , x = '일자', y = '중식계')
plt.show()

매년 연초에는 상승하다가 중반기에 떨어지고 다시 상승하다가 연말에 떨어지는 형태를 보임


In [ ]:
plt.figure(figsize=(15, 12))
sns.barplot(data = train , x = '요일' , y = '중식계')
plt.show()

In [ ]:
plt.figure(figsize=(15, 12))
sns.barplot(data = train , x = '요일' , y = '석식계')
plt.show()

월요일이 사람이 가장 많고 점점 줄다가 금요일에는 가장 적어진다.

# 시간 흐름에 따른 정원 수

In [ ]:
plt.figure(figsize=(12, 8))
sns.lineplot(data = train_mon , x = '일자', y = '본사정원수',label = '정원 수')

In [ ]:
plt.figure(figsize=(15, 12))
sns.lineplot(data = train_mon , x = '일자', y = '본사휴가자수',label = '휴가자 수')
sns.lineplot(data = train_mon , x = '일자', y = '본사출장자수',label = '출장자 수')
sns.lineplot(data = train_mon , x = '일자', y = '현본사소속재택근무자수',label = '재택근무자 수')
plt.show()

매년 휴가자 수는 매년 비슷한 형태로 반복되고 출장자수는 규칙이 없다.
재택근무자수는 2020년 이후 폭발적으로 늘었다.

In [ ]:
sns.barplot(data = train , x = '사회적거리두기 2단계 이상',y = '중식계')


# 중식 메뉴와 석식 메뉴 분석
https://dacon.io/competitions/official/235743/codeshare/2986?page=1&dtype=recent 참고

In [ ]:
train

In [ ]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch_train) 만들기 
lunch_train = []
for day in range(len(train)):
    tmp = train.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch_train.append(tmp) 

In [ ]:
# lunch train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
menu_len_list = []
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = [];
for i, day_menu in enumerate(lunch_train):
    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
    banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
    banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

    if i < 1067:
        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        side_tmp = day_menu[-2]; side.append(side_tmp)
    else: 
        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        side_tmp  = day_menu[-1]; side.append(side_tmp)
    menu_len_list.append([len(day_menu),i])

In [ ]:
train_ln = train.copy()

train_ln['밥'] = bob
train_ln['국'] = gook
train_ln['반찬1'] = banchan1; 
train_ln['반찬2'] = banchan2;
train_ln['반찬3'] = banchan3
train_ln['김치'] = kimchi
train_ln['사이드'] = side

In [ ]:
train_ln.head(5)

저녁 메뉴 분석

In [ ]:
# 일별 저녁메뉴를 작은 리스트로 갖고 있는 큰 리스트 (dinner_train) 만들기 
dinner_train = []
for day in range(len(train)):
    tmp = train.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner_train.append(tmp) 

In [ ]:
# dinner train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = []
for i, day_menu in enumerate(dinner_train):
    if (len(day_menu) < 4 ):
        bob.append('*')
        gook.append('*')
        banchan1.append('*')
        banchan2.append('*')
        banchan3.append('*')

        if i < 1067:
            kimchi.append('*')
            side.append('*')
        else: 
            kimchi.append('*')
            side.append('*')

    elif (len(day_menu)==4):
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3.append('*')

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)
    
    else :
        bob_tmp = day_menu[0]; bob.append(bob_tmp)
        gook_tmp = day_menu[1]; gook.append(gook_tmp)
        banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
        banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
        banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

        if i < 1067:
            kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
            side_tmp = day_menu[-2]; side.append(side_tmp)
        else: 
            kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
            side_tmp  = day_menu[-1]; side.append(side_tmp)

In [ ]:
train_dn = train.copy()

train_dn['밥'] = bob
train_dn['국'] = gook
train_dn['반찬1'] = banchan1; train_dn['반찬2'] = banchan2;
train_dn['반찬3'] = banchan3
train_dn['김치'] = kimchi
train_dn['사이드'] = side

In [ ]:
main30_t = train_ln['반찬1'].value_counts()[:30].to_frame().reset_index()

In [ ]:
target = list(main30_t[0:30]['index'])
target
main30 = train_ln[ train_ln['반찬1'].isin(target)] # top30 메뉴가 나온 날만 뽑기

In [ ]:
main30

In [ ]:
plt.figure(figsize=(20, 15))
sns.barplot(data = main30, x = '반찬1' , y = '중식계')
plt.xticks(rotation=45)
plt.show()

각 메뉴별 식사 인원 수


**가장 많은 인원이 점심을 먹엇을 때의 메뉴** 

메뉴의 공통점이나 5일 사이의 공통점을 좀 확인해보면 좋을 것 같습니다



In [ ]:
train_ln.sort_values('중식계',ascending= False).head(5)

이후 메뉴를 카테고리화 시켜서 그냥 모델에 돌리기

In [ ]:
!pip install konlpy wordcloud

In [ ]:
from PIL import Image
from wordcloud import WordCloud

import numpy as np

# 데이터 전처리

사용하지 않을 칼럼 제거! 출장자수와 휴가자수는 식사 인원 수를 사용하므로 제거 해주었습니당

In [ ]:
train_ln.drop(columns = ['본사정원수','본사휴가자수','본사출장자수','조식메뉴','중식메뉴','석식메뉴','석식계','현본사소속재택근무자수'],inplace = True)

단위가 흠 ,,, 다 같은 인원수라서 필요 없을 것 같지만.. 혹시 모르니까 스케일링은 진행 했습니다. 그래서 스케일링 한 것 과 안한 것의 차이를 좀 뒤에서 보는느낌..

In [ ]:
train_ln['요일'] =  train_ln['요일'].astype('category')
train_ln['요일'] = train_ln['요일'].cat.codes


train_ln['밥'] =  train_ln['밥'].astype('category')
train_ln['밥'] = train_ln['밥'].cat.codes

train_ln['국'] =  train_ln['국'].astype('category')
train_ln['국'] = train_ln['국'].cat.codes

train_ln['반찬1'] =  train_ln['반찬1'].astype('category')
train_ln['반찬1'] = train_ln['반찬1'].cat.codes

train_ln['반찬2'] =  train_ln['반찬2'].astype('category')
train_ln['반찬2'] = train_ln['반찬2'].cat.codes

train_ln['반찬3'] =  train_ln['반찬3'].astype('category')
train_ln['반찬3'] = train_ln['반찬3'].cat.codes

train_ln['김치'] =  train_ln['김치'].astype('category')
train_ln['김치'] = train_ln['김치'].cat.codes

train_ln['사이드'] =  train_ln['사이드'].astype('category')
train_ln['사이드'] = train_ln['사이드'].cat.codes

In [ ]:
train_ln.info()

In [ ]:
train_ln_scaled = train_ln.copy()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_ln_scaled[['본사시간외근무명령서승인건수','식사 인원 수']] = scaler.fit_transform(train_ln_scaled[['본사시간외근무명령서승인건수','식사 인원 수']])

In [ ]:
# 타겟과 관계가 높은 변수들 확인
plt.figure(figsize=(8, 5))
train_ln.corr()['중식계'].drop('중식계').sort_values().plot(kind='barh')

요일, 야근하는 인원 수, 사회적 거리두기 2단계 변수들이 높게 나온 것을 알 수 있습니다.. 맞나여..

야근 할만큼 일이 많고 바빠서 그냥 사내식당에서 많이 먹었나..?

그리고 대한민국이라,, 김치가 높게 나온 것 같습니다..

In [ ]:
plt.figure(figsize = (15,12))
sns.heatmap(train_ln.corr(),cmap = 'Blues')
plt.show()

신기하게 반찬 3 와 사이드가 상관관계가 높은 것을 알 수 있습니다.
이는 사이드 메뉴가 나오지 않을 경우 반찬 3와 같은 것으로 들어가서 그렇습니다..!

In [ ]:
test['식사 인원 수'] = test['본사정원수']- test['본사휴가자수'] - test['본사출장자수'] - test['현본사소속재택근무자수']
test['일자'] = pd.to_datetime(test['일자'])

for i in range(len(test)):
    if test['일자'][i]<datetime.datetime(2020,8,23):
        test.loc[i,'사회적거리두기 1단계']=0
        test.loc[i,'사회적거리두기 1.5단계']=0
        test.loc[i,'사회적거리두기 2단계 이상']=0
    elif datetime.datetime(2020,8,23)<=test['일자'][i]<datetime.datetime(2020,10,13):
        test.loc[i,'사회적거리두기 1단계']=0
        test.loc[i,'사회적거리두기 1.5단계']=0
        test.loc[i,'사회적거리두기 2단계 이상']=1
    elif datetime.datetime(2020,10,13)<=test['일자'][i]<datetime.datetime(2020,12,8):
        test.loc[i,'사회적거리두기 1단계']=1
        test.loc[i,'사회적거리두기 1.5단계']=0
        test.loc[i,'사회적거리두기 2단계 이상']=0
    elif datetime.datetime(2020,12,8)<=test['일자'][i]<datetime.datetime(2021,2,15):
        test.loc[i,'사회적거리두기 1단계']=0
        test.loc[i,'사회적거리두기 1.5단계']=0
        test.loc[i,'사회적거리두기 2단계 이상']=1
    elif datetime.datetime(2021,2,15)<=test['일자'][i]:
        test.loc[i,'사회적거리두기 1단계']=0
        test.loc[i,'사회적거리두기 1.5단계']=1
        test.loc[i,'사회적거리두기 2단계 이상']=0

In [ ]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch_train) 만들기 
lunch_train = []
for day in range(len(test)):
    tmp = test.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch_train.append(tmp)

# lunch train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
menu_len_list = []
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = [];
for i, day_menu in enumerate(lunch_train):
    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
    banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
    banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

    if i < 1067:
        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        side_tmp = day_menu[-2]; side.append(side_tmp)
    else: 
        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        side_tmp  = day_menu[-1]; side.append(side_tmp)
    menu_len_list.append([len(day_menu),i])

test_ln = test.copy()

test_ln['밥'] = bob
test_ln['국'] = gook
test_ln['반찬1'] = banchan1; 
test_ln['반찬2'] = banchan2;
test_ln['반찬3'] = banchan3
test_ln['김치'] = kimchi
test_ln['사이드'] = side

In [ ]:
test_ln['요일'] =  test_ln['요일'].astype('category')
test_ln['요일'] = test_ln['요일'].cat.codes


test_ln['밥'] =  test_ln['밥'].astype('category')
test_ln['밥'] = test_ln['밥'].cat.codes

test_ln['국'] =  test_ln['국'].astype('category')
test_ln['국'] = test_ln['국'].cat.codes

test_ln['반찬1'] =  test_ln['반찬1'].astype('category')
test_ln['반찬1'] = test_ln['반찬1'].cat.codes

test_ln['반찬2'] =  test_ln['반찬2'].astype('category')
test_ln['반찬2'] = test_ln['반찬2'].cat.codes

test_ln['반찬3'] =  test_ln['반찬3'].astype('category')
test_ln['반찬3'] = test_ln['반찬3'].cat.codes

test_ln['김치'] =  test_ln['김치'].astype('category')
test_ln['김치'] = test_ln['김치'].cat.codes

test_ln['사이드'] =  test_ln['사이드'].astype('category')
test_ln['사이드'] = test_ln['사이드'].cat.codes

In [ ]:
test_ln.drop(columns = ['본사정원수','본사휴가자수','본사출장자수','조식메뉴','중식메뉴','석식메뉴','현본사소속재택근무자수'],inplace = True)

# 모델링

In [ ]:
train_ln_scaled.info()

In [ ]:
#best1 = compare_models(fold=5, n_select=2, exclude=['svm','ridge'])

In [ ]:
#tuned_best = tune_model(best1, n_iter = 3)

In [ ]:
#pred = predict_model(tuned_best,test_ln)

In [ ]:
#test['predict'] = pred

In [ ]:
train_ln

일단 적당히 예측만 해보았는데... 다음번까지 직접 시계열모델로 구현하고 좀 더 성능을 높혀서 구현해보도록하겠습니다... 죄송합니당..ㅠㅠ

In [ ]:
train_ln

In [ ]:
# 라이브러리 설치
!pip install delayed

In [ ]:
# 수치형 설명 변수를 활용한 선형 회귀 모형 적합
from sklearn.linear_model import LinearRegression


In [ ]:
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
#  회귀모형 재적합
linear_model = LinearRegression()
    
X = train_ln.drop(columns=['일자','중식계'])
y = train_ln['중식계']

linear_model.fit(X, y)

print(linear_model.intercept_)
print(linear_model.coef_)

In [ ]:
# 회귀 계수 정리
coeff = pd.DataFrame(linear_model.coef_, X.columns,columns=['Coefficient'])
coeff

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
X = train_ln.drop(columns=['일자','중식계'])
y = train_ln['중식계']

X_train, X_test , y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state = 11)

In [ ]:
xgb = XGBRegressor(n_estimators= 500 , learning_rate= 0.1 , max_depth= 4)
xgb.fit(X_train,y_train)
xgb_pred = xgb.predict(X_test)

In [ ]:
mean_absolute_error(y_test, xgb_pred)

In [ ]:
xgb_param_grid = {
    'n_estimators' : [100,200,400,600],
    'learning_rate' : [0.01 , 0.05 , 0.1,0.15,0.2],
    'max_depth' : [4 , 6 , 8, 10 , 12]
}

xgb_grid = GridSearchCV(xgb , param_grid= xgb_param_grid , scoring= 'neg_mean_absolute_error',n_jobs= -1 , verbose= 1)
xgb_grid.fit(X_train,y_train)

In [ ]:
print('최고의 평균 정확도 : {0: 4f}'.format(xgb_grid.best_score_))
print('최고의 파라미터 : ',xgb_grid.best_params_)

In [ ]:
from xgboost import plot_importance
fig, ax = plt.subplots(figsize = (15,12))
plot_importance(xgb, ax = ax)
plt.show()

In [ ]:
train_ln['pred'] = xgb_grid.predict(X)
train_ln

In [ ]:
test_final = test_ln.drop(columns='일자')

In [ ]:
submission['중식계'] = xgb_grid.predict(test_final)

In [ ]:
train_dn

In [ ]:
train_dn.drop(columns = ['본사정원수','본사휴가자수','본사출장자수','조식메뉴','중식메뉴','석식메뉴','현본사소속재택근무자수'],inplace = True)

In [ ]:
train_dn

In [ ]:
train_dn['요일'] =  train_dn['요일'].astype('category')
train_dn['요일'] = train_dn['요일'].cat.codes

train_dn['밥'] =  train_dn['밥'].astype('category')
train_dn['밥'] = train_dn['밥'].cat.codes

train_dn['국'] =  train_dn['국'].astype('category')
train_dn['국'] = train_dn['국'].cat.codes

train_dn['반찬1'] =  train_dn['반찬1'].astype('category')
train_dn['반찬1'] = train_dn['반찬1'].cat.codes

train_dn['반찬2'] =  train_dn['반찬2'].astype('category')
train_dn['반찬2'] = train_dn['반찬2'].cat.codes

train_dn['반찬3'] =  train_dn['반찬3'].astype('category')
train_dn['반찬3'] = train_dn['반찬3'].cat.codes

train_dn['김치'] =  train_dn['김치'].astype('category')
train_dn['김치'] = train_dn['김치'].cat.codes

train_dn['사이드'] =  train_dn['사이드'].astype('category')
train_dn['사이드'] = train_dn['사이드'].cat.codes

In [ ]:
X = train_dn.drop(columns=['일자','중식계','석식계'])
y = train_dn['석식계']

X2_train, X2_test , y2_train, y2_test = train_test_split(X,y,test_size = 0.2,random_state = 11)

In [ ]:
xgb1 = XGBRegressor(n_estimators= 500 , learning_rate= 0.1 , max_depth= 4)

In [ ]:
xgb_param_grid = {
    'n_estimators' : [100,200,400,600],
    'learning_rate' : [0.01 , 0.05 , 0.1,0.15,0.2],
    'max_depth' : [4 , 6 , 8, 10 , 12]
}

xgb_grid1 = GridSearchCV(xgb1 , param_grid= xgb_param_grid , scoring= 'neg_mean_absolute_error',n_jobs= -1 , verbose= 1)
xgb_grid1.fit(X_train,y_train)

In [ ]:
print('최고의 평균 정확도 : {0: 4f}'.format(xgb_grid1.best_score_))
print('최고의 파라미터 : ',xgb_grid1.best_params_)

In [ ]:
fig, ax = plt.subplots(figsize = (15,12))
plot_importance(xgb1, ax = ax)
plt.show()

In [ ]:
test

In [ ]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch_train) 만들기 
lunch_train = []
for day in range(len(test)):
    tmp = test.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch_train.append(tmp)

# lunch train data에 메뉴명별 칼럼 만들기 (밥, 국, 반찬1-3)
menu_len_list = []
bob = []; gook = []; banchan1 = []; banchan2 = []; banchan3 = []; kimchi = []; side = [];
for i, day_menu in enumerate(lunch_train):
    bob_tmp = day_menu[0]; bob.append(bob_tmp)
    gook_tmp = day_menu[1]; gook.append(gook_tmp)
    banchan1_tmp = day_menu[2]; banchan1.append(banchan1_tmp)
    banchan2_tmp = day_menu[3]; banchan2.append(banchan2_tmp)
    banchan3_tmp = day_menu[4]; banchan3.append(banchan3_tmp)

    if i < 1067:
        kimchi_tmp = day_menu[-1]; kimchi.append(kimchi_tmp)
        side_tmp = day_menu[-2]; side.append(side_tmp)
    else: 
        kimchi_tmp = day_menu[-2]; kimchi.append(kimchi_tmp)
        side_tmp  = day_menu[-1]; side.append(side_tmp)
    menu_len_list.append([len(day_menu),i])

test_dn = test.copy()

test_dn['밥'] = bob
test_dn['국'] = gook
test_dn['반찬1'] = banchan1; 
test_dn['반찬2'] = banchan2;
test_dn['반찬3'] = banchan3
test_dn['김치'] = kimchi
test_dn['사이드'] = side

In [ ]:
test_dn.drop(columns = ['본사정원수','본사휴가자수','본사출장자수','조식메뉴','중식메뉴','석식메뉴','현본사소속재택근무자수'],inplace = True)

In [ ]:
test_dn['요일'] =  test_dn['요일'].astype('category')
test_dn['요일'] = test_dn['요일'].cat.codes

test_dn['밥'] =  test_dn['밥'].astype('category')
test_dn['밥'] = test_dn['밥'].cat.codes

test_dn['국'] =  test_dn['국'].astype('category')
test_dn['국'] = test_dn['국'].cat.codes

test_dn['반찬1'] =  test_dn['반찬1'].astype('category')
test_dn['반찬1'] = test_dn['반찬1'].cat.codes

test_dn['반찬2'] =  test_dn['반찬2'].astype('category')
test_dn['반찬2'] = test_dn['반찬2'].cat.codes

test_dn['반찬3'] =  test_dn['반찬3'].astype('category')
test_dn['반찬3'] = test_dn['반찬3'].cat.codes

test_dn['김치'] =  test_dn['김치'].astype('category')
test_dn['김치'] = test_dn['김치'].cat.codes

test_dn['사이드'] =  test_dn['사이드'].astype('category')
test_dn['사이드'] = test_dn['사이드'].cat.codes

In [ ]:
test_dn_final = test_dn.drop(columns=['일자'])

In [ ]:
submission['석식계'] = xgb_grid1.predict(test_dn_final)


In [ ]:
fig, ax = plt.subplots(figsize = (15,12))
plot_importance(xgb, ax = ax)
plt.show()

In [ ]:
submission.to_csv('/content/drive/MyDrive/TAVE 머신러닝 스터디/식수 인원 예측/submission_final_code1.csv', index=False)

In [ ]:
test_dn

In [ ]:
test_ln

In [ ]:
submission

In [ ]:
xgb1.fit(X2_train,y2_train)
xgb1_pred = xgb_grid1.predict(X2_test)

In [ ]:
mean_absolute_error(y2_test, xgb1_pred)